In [1]:
import torch

In [2]:
cos = torch.load('cos')
sin = torch.load('sin')
rotary_pos_emb_causal = torch.load('rotary_pos_emb_causal')


In [44]:
# rotary_pos_emb_flashinfer = torch.cat((cos, sin), dim=1).view(7, 32, 2)
# rotary_pos_emb_flashinfer = torch.cat((cos.squeeze(2), sin.squeeze(2)), dim=2)
rotary_pos_emb_flashinfer = torch.cat((cos.transpose(1, 2), sin.transpose(1, 2)), dim=2)
torch.all(rotary_pos_emb_flashinfer == rotary_pos_emb_causal[0])

tensor(True, device='cuda:0')

In [17]:
key_layer_before_causal = torch.load("key_layer_before_causal")
query_layer_before_causal = torch.load("query_layer_before_causal")
key_layer_before_flashinfer = torch.load("key_layer_before_flashinfer")
query_layer_before_flashinfer = torch.load("query_layer_before_flashinfer")

In [25]:
torch.all(key_layer_before_causal[0] == key_layer_before_flashinfer.transpose(0, 1)), torch.all(query_layer_before_causal[0] == query_layer_before_flashinfer.transpose(0, 1))

(tensor(True, device='cuda:0'), tensor(False, device='cuda:0'))

In [31]:
torch.nonzero(query_layer_before_causal[0] != query_layer_before_flashinfer.transpose(0, 1)).size()

torch.Size([5, 3])

In [27]:
key_layer_after_causal = torch.load("key_layer_after_causal")
query_layer_after_causal = torch.load("query_layer_after_causal")
key_layer_after_flashinfer = torch.load("key_layer_after_flashinfer")
query_layer_after_flashinfer = torch.load("query_layer_after_flashinfer")

In [28]:
torch.all(key_layer_after_causal[0] == key_layer_after_flashinfer.transpose(0, 1)), torch.all(query_layer_after_causal[0] == query_layer_after_flashinfer.transpose(0, 1))

(tensor(False, device='cuda:0'), tensor(False, device='cuda:0'))

In [30]:
torch.nonzero(query_layer_after_causal[0] != query_layer_after_flashinfer.transpose(0, 1)).size()

torch.Size([9464, 3])

In [32]:
torch.nonzero(key_layer_after_causal[0] != key_layer_after_flashinfer.transpose(0, 1)).size()

torch.Size([604, 3])

In [47]:
def assert_close(a, b, rtol, atol):
    # rtol, atol = {
    #     torch.float16: (1e-2, 5e-2),
    #     torch.bfloat16: (8e-3, 8e-3),
    # }[a.dtype]
    torch.testing.assert_close(a, b, rtol=rtol, atol=atol)

In [48]:
assert_close(query_layer_after_causal[0], query_layer_after_flashinfer.transpose(0, 1), 1e-2, 3e-3)

AssertionError: Tensor-likes are not close!

Mismatched elements: 5873 / 28672 (20.5%)
Greatest absolute difference: 11.265625 at index (23, 6, 34) (up to 0.003 allowed)
Greatest relative difference: 928.0 at index (26, 4, 11) (up to 0.01 allowed)

In [49]:
assert_close(query_layer_before_causal[0], query_layer_before_flashinfer.transpose(0, 1), 1e-2, 3e-3)

In [50]:
rotary_pos_emb_causal.shape

torch.Size([1, 7, 32, 2])

In [51]:
torch.select(rotary_pos_emb_causal, dim=3, index=0).shape

torch.Size([1, 7, 32])